In [1]:
# Import steering angle
import numpy as np
import csv
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import cv2
from io import BytesIO
import scipy.misc
import pandas as pd  

lines = []
with open('driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
for line in lines:
    for i in range(3):
        source_path = line[i]
        image = cv2.imread(source_path)
        images.append(image)
    correction = 0.25
    measurement = float(line[3])
    measurements.append(measurement)
    measurements.append(measurement+correction)
    measurements.append(measurement-correction)
    
    
print(len(images))
print(len(measurements))

augmented_images = []
augmented_measurements = []
for image, measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    flipped_image = cv2.flip(image,1)
    flipped_measurement = float(measurement) * -1.0
    augmented_images.append(flipped_image)
    augmented_measurements.append(flipped_measurement)

print(len(augmented_images))
print(len(augmented_measurements))
print('Images loaded')


# TODO: Load the feature data to the variable X_train
X_train = np.array(augmented_images)

# TODO: Load the label data to the variable y_train
y_train = np.array(augmented_measurements)

print(X_train.shape)
print(y_train.shape)

print('Data variables sett')


# TODO: Build a model
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

pool_size = [2,2]
input_shape = [160, 320, 3]
batch_size = 128
nb_epoch = 7

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=input_shape))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,5,5, subsample=pool_size, activation='relu'))
model.add(Convolution2D(36,5,5, subsample=pool_size, activation='relu'))
model.add(Convolution2D(48,5,5, subsample=pool_size, activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile('adam', loss='mse')

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,verbose=1, validation_split=0.2, shuffle=True) 
#model.fit(X_train, y_train, nb_epoch=nb_epoch, validation_split=0.2, shuffle=True) 


model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')
model.summary()




print('done')

25890
25890
51780
51780
Images loaded
(51780, 160, 320, 3)
(51780,)
Data variables sett


Using TensorFlow backend.


Train on 41424 samples, validate on 10356 samples
Epoch 1/7
41424/41424 [==============================] - 1317s - loss: 0.0225 - val_loss: 0.0731